# XLM-RoBERTa Large Text Classifier for Sinhala AI/Human Detection

Text classification model using **XLM-RoBERTa Large** for detecting AI-generated vs Human-written Sinhala text. XLM-RoBERTa is specifically designed for multilingual tasks and performs excellently with low-resource languages like Sinhala.

## 1. Install Required Libraries

In [ ]:
%pip install -q tf-keras
%pip install -q transformers
%pip install -q datasets
%pip install -q nltk
%pip install -q scikit-learn
%pip install -q matplotlib
%pip install -q sentencepiece

## 2. Import Required Libraries

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import re
import warnings
warnings.filterwarnings('ignore')

# Suppress TensorFlow deprecation warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score, classification_report

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 3. Load Dataset from JSONL Files

In [ ]:
def load_jsonl(filepath):
    """Load data from JSONL file"""
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return data

# Load training, validation, and test datasets
train_data = load_jsonl('dataset/train.jsonl')
val_data = load_jsonl('dataset/val.jsonl')
test_data = load_jsonl('dataset/test.jsonl')

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Testing set size: {len(test_data)}")

## 4. Convert JSONL to DataFrame and Preprocess

In [ ]:
# Convert to DataFrame
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Display sample data
print("Sample training data:")
print(train_df[['text', 'label']].head())
print(f"\nLabel value counts (Train):")
print(train_df['label'].value_counts())

## 5. Map Labels to Numeric Values

In [ ]:
# Create label mapping
label_mapping = {'HUMAN': 0, 'AI': 1}
reverse_mapping = {v: k for k, v in label_mapping.items()}

# Map labels to numeric values
train_df['label_encoded'] = train_df['label'].map(label_mapping)
val_df['label_encoded'] = val_df['label'].map(label_mapping)
test_df['label_encoded'] = test_df['label'].map(label_mapping)

# Check for any unmapped values
print(f"Train - Unmapped labels: {train_df['label_encoded'].isna().sum()}")
print(f"Val - Unmapped labels: {val_df['label_encoded'].isna().sum()}")
print(f"Test - Unmapped labels: {test_df['label_encoded'].isna().sum()}")

## 6. Text Preprocessing

XLM-RoBERTa handles multilingual text well, but we'll still apply Sinhala-specific normalization for consistency.

In [ ]:
import unicodedata

# Sinhala-focused normalization & contraction expansion

def expand_contractions(text):
    """
    Normalize Sinhala text and expand a small set of common colloquial contractions.
    - NFC Unicode normalization (avoids mixed composition forms)
    - Remove Zero-Width Joiner/Non-Joiner which can appear in Sinhala typing
    - Expand a curated list of common colloquial forms to their standard forms
    Note: This list is intentionally conservative to avoid changing semantics.
    """
    if not isinstance(text, str):
        return text

    # Normalize to NFC to standardize diacritics
    text = unicodedata.normalize('NFC', text)

    # Remove zero-width characters often accidentally present
    text = text.replace('\u200c', '').replace('\u200d', '')  # ZWNJ/ZWJ

    # Conservative Sinhala "contractions" / colloquial expansions
    contractions_si = {
        # Negation forms
        'නෑ': 'නැහැ',
        'බෑ': 'බැහැ',
        # Colloquial vowels / emphasis
        'දැං': 'දැන්',
        'කොහේද': 'කොහෙද',
        'මොකෝ': 'මොකද',
        # Common short forms ending with "ෙ" to "ේ" (targeted words only)
        'ඇතුලෙ': 'ඇතුලේ',
        'බාහිරෙ': 'බාහිරේ',
        'වැඩෙ': 'වැඩේ',
        'රජයෙ': 'රජයේ',
        'යාලුවෙ': 'යාලුවේ',
    }

    # Apply word-boundary replacements for items above
    for contraction, expanded in contractions_si.items():
        text = re.sub(r"\b" + re.escape(contraction) + r"\b", expanded, text)

    # Targeted possessive expansions (avoid generic "ගෙ" → "ගේ" to not corrupt 'ගෙදර')
    text = re.sub(r"\bමගෙ\b", "මගේ", text)
    text = re.sub(r"\bඔයාගෙ\b", "ඔයාගේ", text)
    text = re.sub(r"\bඔගෙ\b", "ඔගේ", text)

    return text

# Apply expansion of contractions using Sinhala-focused normalization
print("Preprocessing training data (Sinhala normalization)...")
train_df['expanded_text'] = train_df['text'].apply(expand_contractions)

print("Preprocessing validation data (Sinhala normalization)...")
val_df['expanded_text'] = val_df['text'].apply(expand_contractions)

print("Preprocessing test data (Sinhala normalization)...")
test_df['expanded_text'] = test_df['text'].apply(expand_contractions)

print("Preprocessing complete!")

## 7. Load XLM-RoBERTa Tokenizer

XLM-RoBERTa is trained on 100 languages including Sinhala. We'll use the base model (~560MB) which provides excellent multilingual performance.

In [ ]:
# Load XLM-RoBERTa Large tokenizer
model_name = 'xlm-roberta-large'  # ~2.2GB, 550M parameters, best multilingual performance

tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"XLM-RoBERTa Large tokenizer loaded successfully from {model_name}!")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

## 8. Tokenize and Encode Text Data

Using **max_length=256** for better generalization and larger effective batch size.

In [ ]:
# Tokenize and encode the text data using tokenizer in batches to avoid memory spikes
print("Tokenizing data in batches...")

MAX_LENGTH = 256  # Optimal length for better generalization and GPU efficiency

def tokenize_in_batches(texts, tokenizer, batch_size=32, max_length=MAX_LENGTH):
    """Tokenize a list of texts in small batches and return numpy arrays.
    Returns a dict with 'input_ids' and 'attention_mask' as numpy arrays.
    Pads every batch to `max_length` so concatenation shapes match.
    """
    input_ids_parts = []
    attention_mask_parts = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            padding='max_length',   # pad each batch to the fixed max_length
            truncation=True,
            max_length=max_length,
            return_tensors='tf'
        )
        # Convert to numpy to keep memory usage predictable and avoid nested lists
        ids = enc['input_ids'].numpy()
        mask = enc['attention_mask'].numpy()

        # Sanity check: ensure shape[1] == max_length
        if ids.shape[1] != max_length:
            # If tokenizer produced a different length for some reason, force-pad/truncate
            ids = np.pad(ids, ((0,0),(0,max_length-ids.shape[1])), constant_values=0)[:,:max_length]
            mask = np.pad(mask, ((0,0),(0,max_length-mask.shape[1])), constant_values=0)[:,:max_length]

        input_ids_parts.append(ids)
        attention_mask_parts.append(mask)

    input_ids = np.concatenate(input_ids_parts, axis=0)
    attention_mask = np.concatenate(attention_mask_parts, axis=0)

    return {'input_ids': input_ids, 'attention_mask': attention_mask}

# Run batched tokenization
train_encodings = tokenize_in_batches(train_df['expanded_text'].tolist(), tokenizer, batch_size=32)
val_encodings = tokenize_in_batches(val_df['expanded_text'].tolist(), tokenizer, batch_size=32)
test_encodings = tokenize_in_batches(test_df['expanded_text'].tolist(), tokenizer, batch_size=32)

print("Tokenization complete!")
print(f"Training encodings shape: {train_encodings['input_ids'].shape}")
print(f"Validation encodings shape: {val_encodings['input_ids'].shape}")
print(f"Test encodings shape: {test_encodings['input_ids'].shape}")

## 9. Prepare Labels

In [ ]:
# Extract input arrays and convert labels to numpy arrays
# Use dicts with both input_ids and attention_mask so the model gets both inputs
train_inputs = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask']
}
val_inputs = {
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask']
}
test_inputs = {
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
}

# Convert labels to numpy arrays
train_labels = np.array(train_df['label_encoded'].astype(int).tolist())
val_labels = np.array(val_df['label_encoded'].astype(int).tolist())
test_labels = np.array(test_df['label_encoded'].astype(int).tolist())

print(f"Train labels shape: {train_labels.shape}")
print(f"Val labels shape: {val_labels.shape}")
print(f"Test labels shape: {test_labels.shape}")
print(f"\nLabel distribution (Train): {np.bincount(train_labels)}")
print(f"Label distribution (Val): {np.bincount(val_labels)}")
print(f"Label distribution (Test): {np.bincount(test_labels)}")

## 10. Define XLM-RoBERTa Classification Model

Loading the pre-trained XLM-RoBERTa model for sequence classification.

In [ ]:
# Load pre-trained XLM-RoBERTa Large model for sequence classification
print("Loading XLM-RoBERTa Large model (this may take a moment)...")

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    hidden_dropout_prob=0.2,
    attention_probs_dropout_prob=0.2
)

print(f"XLM-RoBERTa Large model loaded successfully from {model_name}!")
print(f"Model has {model.num_parameters():,} parameters")

## 11. Compile the Model with Optimized Settings

Using:
- **Learning rate:** 1.5e-5 (optimal for Large model)
- **Weight decay:** 0.01 (L2 regularization)
- **Gradient clipping:** 1.0 (prevents exploding gradients)
- **Mixed precision:** fp16 (faster training on modern GPUs)

In [ ]:
# Enable mixed precision training for faster training (requires compatible GPU)
try:
    from tensorflow.keras import mixed_precision
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Mixed precision (fp16) enabled!")
except:
    print("Mixed precision not available, using default precision")

# Compile the model with optimized hyperparameters for XLM-RoBERTa Large
from transformers import create_optimizer

num_train_steps = len(train_labels) // 16 * 6
num_warmup_steps = int(0.08 * num_train_steps)

optimizer, lr_schedule = create_optimizer(
    init_lr=1.5e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    weight_decay_rate=0.01
)

optimizer.clipnorm = 1.0

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[metric]
)

print("Model compiled successfully!")
print(f"Total training steps: {num_train_steps}")
print(f"Warmup steps: {num_warmup_steps}")
print(f"Learning rate: 1.5e-5")
print(f"Weight decay: 0.01")
print(f"Gradient clipping: 1.0")

print("\nModel Summary:")
model.summary()

## 12. Calculate Class Weights and Train the Model

Using:
- **Class weights** to improve AI recall (addresses class imbalance)
- **Early stopping** with patience=2 to prevent overfitting
- **Gradient accumulation** (effective batch size = 32)
- **6 epochs** with validation monitoring

In [ ]:
# Calculate class weights to improve AI recall
from sklearn.utils.class_weight import compute_class_weight

class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weight = {
    0: class_weights_array[0],
    1: class_weights_array[1] * 1.2
}

print(f"Class weights: {class_weight}")
print(f"This helps the model focus more on the AI class to improve recall.\n")

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=1e-7,
    verbose=1
)

print("Training the XLM-RoBERTa Large model...")
print("This may take some time depending on your hardware.\n")

history = model.fit(
    train_inputs,
    train_labels,
    epochs=6,
    batch_size=16,
    validation_data=(val_inputs, val_labels),
    class_weight=class_weight,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nTraining completed!")
print(f"Best epoch: {len(history.history['loss']) - early_stopping.patience if early_stopping.stopped_epoch > 0 else len(history.history['loss'])}")

## 13. Save the Trained Model

In [ ]:
# Create directory if it doesn't exist
import os
os.makedirs("models/xlm_roberta_classifier", exist_ok=True)

# Save the entire trained model
model.save_pretrained("models/xlm_roberta_classifier/")
print("Model saved to models/xlm_roberta_classifier/")

# Also save the tokenizer
tokenizer.save_pretrained("models/xlm_roberta_classifier/")
print("Tokenizer saved to models/xlm_roberta_classifier/")

## 14. Evaluate on Validation Set

In [ ]:
# Evaluate the model on the validation set
print("Evaluating on validation set...")
val_loss, val_accuracy = model.evaluate(val_inputs, val_labels, verbose=0)
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

## 15. Evaluate on Test Set

In [ ]:
# Evaluate the model on the test set
print("Evaluating on test set...")
test_loss, test_accuracy = model.evaluate(test_inputs, test_labels, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

## 16. Tune Decision Threshold for Optimal F1 Score

Instead of using default 0.5 threshold, we'll find the optimal threshold that maximizes F1 score.

In [ ]:
# Get predictions on validation set to tune threshold
print("Tuning decision threshold on validation set...")
val_predictions = model.predict(val_inputs, verbose=0)
val_probabilities = tf.nn.softmax(val_predictions.logits)[:, 1].numpy()

# Try different thresholds and find the one that maximizes F1 score
from sklearn.metrics import f1_score

thresholds_to_try = np.arange(0.3, 0.7, 0.01)
best_f1 = 0
best_threshold = 0.5

for threshold in thresholds_to_try:
    val_preds_at_threshold = (val_probabilities >= threshold).astype(int)
    f1 = f1_score(val_labels, val_preds_at_threshold)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"\nOptimal threshold: {best_threshold:.3f}")
print(f"F1 Score at optimal threshold: {best_f1:.4f}")
print(f"F1 Score at default 0.5: {f1_score(val_labels, (val_probabilities >= 0.5).astype(int)):.4f}")
print(f"\nUsing threshold {best_threshold:.3f} for test set evaluation.")

## 16. Plot Training and Validation Accuracy

In [ ]:
# Create results directory if it doesn't exist
os.makedirs("results", exist_ok=True)

# Plot accuracy and loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, marker='s')
plt.title('XLM-RoBERTa: Training and Validation Accuracy', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2, marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, marker='s')
plt.title('XLM-RoBERTa: Training and Validation Loss', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/xlm_roberta_training_history.png', dpi=100, bbox_inches='tight')
plt.show()

print("Training history plot saved!")

## 17. Plot Training and Validation Accuracy

In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Create results directory if it doesn't exist
os.makedirs("results", exist_ok=True)

# Predict labels for the test set
print("Generating predictions on test set...")
predictions = model.predict(test_inputs)
predicted_labels = np.argmax(predictions.logits, axis=1)

# Generate confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)

# Display confusion matrix
plt.figure(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['HUMAN', 'AI'])
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix - XLM-RoBERTa Large Text Classification', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('results/xlm_roberta_confusion_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

print("Confusion matrix saved!")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, marker='s')
plt.title('XLM-RoBERTa Large: Training and Validation Accuracy', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2, marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, marker='s')
plt.title('XLM-RoBERTa Large: Training and Validation Loss', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/xlm_roberta_large_training_history.png', dpi=100, bbox_inches='tight')
plt.show()

print("Training history plot saved!")

## 18. Generate Predictions with Optimal Threshold

In [ ]:
# Get probabilities for positive class (AI)
probabilities = tf.nn.softmax(predictions.logits)[:, 1]

# Calculate AUC score
auc_score = roc_auc_score(test_labels, probabilities)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, probabilities)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'XLM-RoBERTa (AUC = {auc_score:.4f})', linewidth=2, color='darkgreen')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=11)
plt.ylabel('True Positive Rate', fontsize=11)
plt.title('ROC Curve - XLM-RoBERTa Text Classification', fontsize=12, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/xlm_roberta_roc_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"AUC Score: {auc_score:.4f}")

# Predict labels for the test set using optimal threshold
print("Generating predictions on test set with optimal threshold...")
test_predictions = model.predict(test_inputs, verbose=0)
test_probabilities = tf.nn.softmax(test_predictions.logits)[:, 1].numpy()

# Use optimal threshold instead of default 0.5
predicted_labels = (test_probabilities >= best_threshold).astype(int)

print(f"Using threshold: {best_threshold:.3f}")
print(f"Predictions generated!\n")

# Also get predictions at default threshold for comparison
predicted_labels_default = np.argmax(test_predictions.logits, axis=1)

# Generate confusion matrices
cm = confusion_matrix(test_labels, predicted_labels)
cm_default = confusion_matrix(test_labels, predicted_labels_default)

# Display confusion matrices side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

disp1 = ConfusionMatrixDisplay(confusion_matrix=cm_default, display_labels=['HUMAN', 'AI'])
disp1.plot(cmap=plt.cm.Blues, values_format='d', ax=ax1)
ax1.set_title('Confusion Matrix - Default Threshold (0.5)', fontsize=12, fontweight='bold')

disp2 = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['HUMAN', 'AI'])
disp2.plot(cmap=plt.cm.Greens, values_format='d', ax=ax2)
ax2.set_title(f'Confusion Matrix - Optimal Threshold ({best_threshold:.3f})', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('results/xlm_roberta_large_confusion_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

print("Confusion matrices saved!")

## 19. Calculate ROC Curve and AUC Score

In [ ]:
# Generate detailed classification reports for both thresholds
report_optimal = classification_report(
    test_labels,
    predicted_labels,
    target_names=['HUMAN', 'AI'],
    digits=4
)

report_default = classification_report(
    test_labels,
    predicted_labels_default,
    target_names=['HUMAN', 'AI'],
    digits=4
)

print("\n" + "="*70)
print("CLASSIFICATION REPORT - DEFAULT THRESHOLD (0.5)")
print("="*70)
print(report_default)
print("="*70)

print("\n" + "="*70)
print(f"CLASSIFICATION REPORT - OPTIMAL THRESHOLD ({best_threshold:.3f})")
print("="*70)
print(report_optimal)
print("="*70)

# Calculate per-class metrics for comparison
from sklearn.metrics import precision_recall_fscore_support

precision_opt, recall_opt, f1_opt, _ = precision_recall_fscore_support(
    test_labels, predicted_labels, average=None, labels=[0, 1]
)

precision_def, recall_def, f1_def, _ = precision_recall_fscore_support(
    test_labels, predicted_labels_default, average=None, labels=[0, 1]
)

print("\n" + "="*70)
print("KEY METRICS COMPARISON")
print("="*70)
print(f"{'Metric':<30} {'Default (0.5)':<20} {'Optimal':<20}")
print("-"*70)
print(f"{'AI Recall (Most Important)':<30} {recall_def[1]:<20.4f} {recall_opt[1]:<20.4f}")
print(f"{'AI Precision':<30} {precision_def[1]:<20.4f} {precision_opt[1]:<20.4f}")
print(f"{'AI F1-Score':<30} {f1_def[1]:<20.4f} {f1_opt[1]:<20.4f}")
print(f"{'Human Recall':<30} {recall_def[0]:<20.4f} {recall_opt[0]:<20.4f}")
print(f"{'Human Precision':<30} {precision_def[0]:<20.4f} {precision_opt[0]:<20.4f}")
print("="*70)

# Calculate AUC score
auc_score = roc_auc_score(test_labels, test_probabilities)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, test_probabilities)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'XLM-RoBERTa Large (AUC = {auc_score:.4f})', linewidth=2, color='darkgreen')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Classifier')

# Mark the optimal threshold point on ROC curve
optimal_idx = np.argmin(np.abs(thresholds - best_threshold))
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=100, zorder=5, 
            label=f'Optimal Threshold ({best_threshold:.3f})')

plt.xlabel('False Positive Rate', fontsize=11)
plt.ylabel('True Positive Rate', fontsize=11)
plt.title('ROC Curve - XLM-RoBERTa Large', fontsize=12, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/xlm_roberta_large_roc_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"AUC Score: {auc_score:.4f}")

## 21. Summary of Model Performance

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracies at both thresholds
accuracy_optimal = accuracy_score(test_labels, predicted_labels)
accuracy_default = accuracy_score(test_labels, predicted_labels_default)

# Create comprehensive summary
summary_data = {
    'Metric': [
        'Model',
        'Parameters',
        'Model Size',
        'Max Sequence Length',
        'Training Epochs',
        'Learning Rate',
        'Effective Batch Size',
        '',
        'Test Accuracy (Default 0.5)',
        'Test Accuracy (Optimal Threshold)',
        'Test Loss',
        'AUC Score',
        'Optimal Threshold',
        '',
        'AI Recall (Default)',
        'AI Recall (Optimal)',
        'AI Precision (Optimal)',
        'AI F1-Score (Optimal)',
        '',
        'Human Recall (Optimal)',
        'Human Precision (Optimal)'
    ],
    'Value': [
        'XLM-RoBERTa Large',
        f'{model.num_parameters():,}',
        '~2.2GB',
        '256 tokens',
        f'{len(history.history["loss"])}',
        '1.5e-5',
        '32 (16 x 2 accumulation)',
        '',
        f'{accuracy_default:.4f}',
        f'{accuracy_optimal:.4f}',
        f'{test_loss:.4f}',
        f'{auc_score:.4f}',
        f'{best_threshold:.3f}',
        '',
        f'{recall_def[1]:.4f}',
        f'{recall_opt[1]:.4f}',
        f'{precision_opt[1]:.4f}',
        f'{f1_opt[1]:.4f}',
        '',
        f'{recall_opt[0]:.4f}',
        f'{precision_opt[0]:.4f}'
    ]
}

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*70)
print("XLM-RoBERTa LARGE MODEL PERFORMANCE SUMMARY")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

summary_df.to_csv('results/xlm_roberta_large_performance_summary.csv', index=False)
print("\nPerformance summary saved to results/xlm_roberta_large_performance_summary.csv")

## 22. Test the Model with Sample Predictions

In [ ]:
def predict_text(text, model, tokenizer, threshold=best_threshold):
    """
    Predict whether a given text is AI-generated or Human-written
    Uses the optimal threshold for classification
    """
    # Preprocess the text
    processed_text = expand_contractions(text)
    
    # Tokenize
    inputs = tokenizer(
        processed_text,
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='tf'
    )
    
    # Predict
    predictions = model.predict({
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }, verbose=0)
    
    # Get probabilities
    probs = tf.nn.softmax(predictions.logits)[0]
    ai_prob = float(probs[1])
    
    # Use optimal threshold
    predicted_label = 1 if ai_prob >= threshold else 0
    
    return {
        'label': reverse_mapping[predicted_label],
        'human_prob': float(probs[0]),
        'ai_prob': ai_prob,
        'confidence': ai_prob if predicted_label == 1 else float(probs[0]),
        'threshold_used': threshold
    }

# Test with sample texts from test set
print("\n" + "="*70)
print(f"SAMPLE PREDICTIONS (Using Optimal Threshold: {best_threshold:.3f})")
print("="*70)

sample_indices = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
for idx in sample_indices:
    if idx < len(test_df):
        text = test_df.iloc[idx]['text']
        true_label = test_df.iloc[idx]['label']
        
        result = predict_text(text, model, tokenizer)
        
        correct = "✓" if result['label'] == true_label else "✗"
        
        print(f"\nText: {text[:100]}...")
        print(f"True Label: {true_label}")
        print(f"Predicted: {result['label']} {correct} (Confidence: {result['confidence']:.2%})")
        print(f"Human Prob: {result['human_prob']:.2%} | AI Prob: {result['ai_prob']:.2%}")
        print("-" * 70)